In [11]:
import polars as pl
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
# Path to the Parquet file
file_path = r"D:\20241207_week5_products_uav_data\output\extract\20241029_week8_project_0_IMG_0002_6.tif.parquet"

# Load the Parquet file into a Polars DataFrame
df = pl.read_parquet(file_path)

# Display the first few rows
max(df["band2"])


0

In [ ]:
"D:\20241207_week5_products_uav_data\output\extract\20241029_week8_project_0_IMG_0000_6.tif.parquet"

In [ ]:
import math

def latlon_to_utm32n_series(lat_deg, lon_deg):
    """
    Convert geographic coordinates (lat, lon in degrees, WGS84)
    to UTM Zone 32N (EPSG:32632) using the standard UTM formulas.

    Returns:
      (easting, northing) in meters.
    """
    # WGS84 ellipsoid constants
    a = 6378137.0                       # semi-major axis (meters)
    f = 1 / 298.257223563               # flattening
    e2 = 2*f - f**2                     # eccentricity squared
    e = math.sqrt(e2)

    # UTM parameters for Zone 32N
    k0 = 0.9996
    E0 = 500000.0                       # false easting
    N0 = 0.0                            # false northing (northern hemisphere)
    lambda0 = math.radians(9.0)         # central meridian for Zone 32N (9°E)

    # Convert input latitude and longitude from degrees to radians
    phi = math.radians(lat_deg)
    lam = math.radians(lon_deg)

    # Compute auxiliary values
    N_val = a / math.sqrt(1 - e2 * math.sin(phi)**2)
    T = math.tan(phi)**2
    # Second eccentricity squared
    ep2 = e2 / (1 - e2)
    C = ep2 * math.cos(phi)**2
    A = (lam - lambda0) * math.cos(phi)

    # Meridional arc length (M)
    M = a * (
          (1 - e2/4 - 3*e2**2/64 - 5*e2**3/256) * phi
        - (3*e2/8 + 3*e2**2/32 + 45*e2**3/1024) * math.sin(2*phi)
        + (15*e2**2/256 + 45*e2**3/1024) * math.sin(4*phi)
        - (35*e2**3/3072) * math.sin(6*phi)
    )

    # Calculate Easting and Northing using standard UTM series formulas
    easting = E0 + k0 * N_val * (
          A
        + (1 - T + C) * A**3 / 6
        + (5 - 18*T + T**2 + 72*C - 58*ep2) * A**5 / 120
    )

    northing = N0 + k0 * (
          M
        + N_val * math.tan(phi) * (
              A**2 / 2
            + (5 - T + 9*C + 4*C**2) * A**4 / 24
            + (61 - 58*T + T**2 + 600*C - 330*ep2) * A**6 / 720
        )
    )

    return easting, northing


# Test the function with a known point

coord = [9.9181978710164600, 51.5649526394502686]
easting, northing = latlon_to_utm32n_series(coord[1], coord[0])



print(f"Latitude: {coord[1]}, Longitude: {coord[0]}")
print(f"Easting: {easting:.2f} m, Northing: {northing:.2f} m")

In [ ]:
df.head(400)

In [ ]:

# Load the GPKG file
file_path =  "/run/media/mak/OS/example_data_week8/20241029_products_uav_data/20241204_oncerco_plot_polygons.gpkg"
gdf = gpd.read_file(file_path)

# Display the first few rows
print(gdf.head())
print(gdf.crs)


In [ ]:
gdf

In [ ]:
print(gdf.shape)  # Should show (number_of_rows, number_of_columns)
print(gdf.isnull().sum())  # Check for missing values
print(gdf.is_valid)  # Should return True for all rows


In [ ]:

# Load camera data (adjust delimiter as needed)
camera_df = pd.read_csv("/run/media/mak/OS/example_data_week8/20241029_products_uav_data/20241029_week8_cameras.txt", sep="\t",  skiprows=2, header=None, )

camera_df.columns = ['PhotoID', 'X', 'Y', 'Z', 'Omega', 'Phi', 'Kappa', 'r11', 'r12', 'r13',
                          'r21', 'r22', 'r23', 'r31', 'r32', 'r33']
# Create a geometry column (assuming X is longitude and Y is latitude)
camera_df['geometry'] = camera_df.apply(lambda row: Point(row['X'], row['Y']), axis=1)

# Create a GeoDataFrame with the camera data
camera_gdf = gpd.GeoDataFrame(camera_df, geometry='geometry', crs="EPSG:4326")

camera_gdf

In [ ]:
camera_in_projected = camera_gdf.to_crs("EPSG:32632")
camera_in_projected

In [ ]:
camera_in_projected.plot(aspect=1)


In [ ]:
gdf["camera points"] = camera_in_projected.geometry
gdf.plot(aspect=1)

In [ ]:
polygon = gdf["geometry"][0]
print(polygon)
print(gdf["geometry"][0])
i=0

for point in camera_in_projected["geometry"]:
    if polygon.contains(point):
        print(point)
        print(camera_in_projected["PhotoID"][i])

    i = i + 1




In [ ]:
polygon = gdf["geometry"][5]
for i in range(len(camera_df)):
    point = Point(latlon_to_utm32n_series(camera_df["Y"][i], camera_df["X"][i]))
   #print(f"function point {camera_in_projected["geometry"][i]} Library poinT: {point}  " )
    if polygon.contains(point):
        print(camera_df["PhotoID"][i])


In [ ]:
from rasterio.transform import from_origin
import rasterio
import logging

def parquet_to_multiband_tif(parquet_path, output_tif, band_columns=["band1", "band2", "band3"],
                             crs="EPSG:32632", nodata=None):
    """
    Reads a Parquet file containing georeferenced data and writes out a multi-band GeoTIFF.

    The Parquet file must include at least the following columns:
        - "Xw": x-coordinate (e.g., projected easting)
        - "Yw": y-coordinate (e.g., projected northing)
    and one or more band columns (e.g., "band1", "band2", "band3").

    Duplicate coordinate entries are aggregated by taking the mean.

    Parameters:
        parquet_path (str): Path to the input Parquet file.
        output_tif (str): Path where the output GeoTIFF will be saved.
        band_columns (list of str): List of column names to export as bands.
        crs (str): Coordinate reference system for the output GeoTIFF.
        nodata: Value to assign for missing data. If None, missing data remains as NaN.

    Returns:
        None. The GeoTIFF is written to output_tif.
    """
    # Read the Parquet file using Polars and convert to a Pandas DataFrame.
    df = pl.read_parquet(parquet_path)
    df_pd = df.to_pandas()

    # Check required coordinate columns
    if not {"Xw", "Yw"}.issubset(df_pd.columns):
        raise ValueError("The input file must contain 'Xw' and 'Yw' coordinate columns.")

    # Get unique coordinates for the grid.
    x_unique = np.sort(df_pd["Xw"].unique())
    # For rasters, Y is sorted in descending order (top-to-bottom).
    y_unique = np.sort(df_pd["Yw"].unique())[::-1]

    if len(x_unique) < 2 or len(y_unique) < 2:
        raise ValueError("Not enough unique coordinate values to form a raster grid.")

    # Calculate pixel size (assumes constant spacing)
    pixel_width = np.round(x_unique[1] - x_unique[0], 6)
    pixel_height = np.round(abs(y_unique[0] - y_unique[1]), 6)

    # Define the affine transform: origin is the top-left corner.
    origin_x = x_unique[0]
    origin_y = y_unique[0]
    transform = from_origin(origin_x, origin_y, pixel_width, pixel_height)

    band_arrays = []
    for band in band_columns:
        if band not in df_pd.columns:
            raise ValueError(f"Band column '{band}' not found in the input file.")
        print(df_pd)
        # Group by coordinates to aggregate duplicate (Xw, Yw) pairs (using mean).
        df_grouped = df_pd.groupby(["Xw", "Yw"], as_index=False)[band].mean()
        # Pivot the grouped data: rows by Yw, columns by Xw.
        pivot = df_grouped.pivot(index="Yw", columns="Xw", values=band)

        # Reindex to ensure all coordinate positions are present in the proper order.
        pivot = pivot.reindex(index=y_unique, columns=x_unique)
        band_array = pivot.values
        print("here")

        # Replace missing values with nodata if provided.
        if nodata is not None:
            band_array = np.where(np.isnan(band_array), nodata, band_array)
        band_arrays.append(band_array)

    # Use the dimensions from the first band.
    height, width = band_arrays[0].shape
    dtype = band_arrays[0].dtype

    logging.info(f"Raster dimensions: width={width}, height={height}")
    logging.info(f"Affine Transform: {transform}")
    logging.info(f"CRS: {crs}")

    # Write the multi-band GeoTIFF.
    with rasterio.open(
         output_tif,
         "w",
         driver="GTiff",
         height=height,
         width=width,
         count=len(band_columns),
         dtype=dtype,
         crs=crs,
         transform=transform,
         nodata=nodata
    ) as dst:
         for idx, band_array in enumerate(band_arrays, start=1):
             dst.write(band_array, idx)

    logging.info(f"GeoTIFF successfully saved to {output_tif}")


In [ ]:
parquet_path = file_path
output_tif = "./output.tif"
bands = ["band1", "band2", "band3"]  # adjust based on your data
parquet_to_multiband_tif(parquet_path, output_tif, bands, crs="EPSG:32632", nodata=0)